# Introduction to Built-in Data Sources



## Prepare the environment
First, we are going to prepare the environment for running PySaprk in the Google Collab Machine (if you work directly in your computer, and you want to prepare it, read and follow champter 2 instructions)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python /content/drive/MyDrive/colab/massive/install_pyspark.py

Install JAVA 8
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=997ea48aa80b7060567eaca589c2ab32a2cddec5754b6f89e2c62e973b8cf2c3
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Obtaining last version of spark
/content/drive/MyDrive/colab/massive/install_pyspark.py:17: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 17 of the file /content/drive/MyDrive/colab/massive/install_pyspark.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html_doc)
Getting version spar

## Start working with Spark
Now we now and understand how Spark appeared in our lives and more or less how it works (and you know, it's amazing 🤭), we can start to work with it.
As you now, the SparkSession is the way programmers "talk" with Spark. So, we need to inicialize that.
TAKE INTO ACCOUNT, this time, qwe are using, also, "enableHiveSupport". This is beacouse we are going to create databases in SparkSQL and it's needed.

In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession
 .builder
 .appName("P2A2")
 .enableHiveSupport()
 .getOrCreate())

spark

## Read data from CSV
Download data from https://raw.githubusercontent.com/databricks/LearningSparkV2/master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv

In [4]:
import wget
# Path to data set
csv_url = "https://raw.githubusercontent.com/databricks/LearningSparkV2/master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
wget.download(csv_url)
# Read and create a temporary view
# Infer schema (note that for larger files you
# may want to specify the schema)
schema = "`date` STRING, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING"

df = (spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .schema(schema)
 .load("departuredelays.csv"))
df.createOrReplaceTempView("us_delay_flights_tbl")


In [5]:
df.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
|01061215|   -6|     602|   ABE|        ATL|
|01061725|   69|     602|   ABE|        ATL|
|01061230|    0|     369|   ABE|        DTW|
|01060625|   -3|     602|   ABE|        ATL|
|01070600|    0|     369|   ABE|        DTW|
|01071725|    0|     602|   ABE|        ATL|
|01071230|    0|     369|   ABE|        DTW|
|01070625|    0|     602|   ABE|        ATL|
|01071219|    0|     569|   ABE|        ORD|
|01080600|

In [6]:
display(df)

DataFrame[date: string, delay: int, distance: int, origin: string, destination: string]

## Using SPARK SQL

### Filter flights whose distance is > 1000 miles

In [ ]:
 ### Filter flights whose distance is > 1000 miles

In [7]:
spark.sql("""
  SELECT distinct DISTANCE , ORIGIN, DESTINATION
  FROM  us_delay_flights_tbl
  WHERE DISTANCE > 1000
  ORDER BY distance DESC
  """).show()

+--------+------+-----------+
|DISTANCE|ORIGIN|DESTINATION|
+--------+------+-----------+
|    4330|   JFK|        HNL|
|    4330|   HNL|        JFK|
|    4312|   HNL|        EWR|
|    4312|   EWR|        HNL|
|    4186|   HNL|        IAD|
|    4186|   IAD|        HNL|
|    3912|   ATL|        HNL|
|    3912|   HNL|        ATL|
|    3687|   HNL|        ORD|
|    3687|   ORD|        HNL|
|    3392|   IAH|        HNL|
|    3392|   HNL|        IAH|
|    3303|   GUM|        HNL|
|    3303|   HNL|        GUM|
|    3288|   HNL|        DFW|
|    3288|   DFW|        HNL|
|    3224|   OGG|        DFW|
|    3224|   DFW|        OGG|
|    2967|   LIH|        DEN|
|    2967|   DEN|        LIH|
+--------+------+-----------+
only showing top 20 rows



### Filter flights whose distance is > 1000 miles

In [8]:
spark.sql("""
  SELECT date, delay, origin, destination
  FROM  us_delay_flights_tbl
  WHERE delay > 120 and ORIGIN = 'SFO'
  AND destination = 'ORD'
  ORDER BY delay DESC
  """).show()

+--------+-----+------+-----------+
|    date|delay|origin|destination|
+--------+-----+------+-----------+
|02190925| 1638|   SFO|        ORD|
|01031755|  396|   SFO|        ORD|
|01022330|  326|   SFO|        ORD|
|01051205|  320|   SFO|        ORD|
|01190925|  297|   SFO|        ORD|
|02171115|  296|   SFO|        ORD|
|01071040|  279|   SFO|        ORD|
|01051550|  274|   SFO|        ORD|
|03120730|  266|   SFO|        ORD|
|01261104|  258|   SFO|        ORD|
|01161210|  225|   SFO|        ORD|
|02091800|  223|   SFO|        ORD|
|01221040|  215|   SFO|        ORD|
|03121155|  203|   SFO|        ORD|
|02111256|  197|   SFO|        ORD|
|03311405|  196|   SFO|        ORD|
|01031920|  193|   SFO|        ORD|
|01021410|  190|   SFO|        ORD|
|03171215|  189|   SFO|        ORD|
|01101410|  184|   SFO|        ORD|
+--------+-----+------+-----------+
only showing top 20 rows



 ## Exercise 1:

*   Convert the DATE column into readable format

In [13]:
spark.sql("""
SELECT
  FROM_UNIXTIME(UNIX_TIMESTAMP(date, 'MMddHHmm'), 'yyyy-MM-dd HH:mm:ss') AS formatted_date,
  delay,
  distance,
  origin,
  destination
FROM us_delay_flights_tbl
""").show()

+-------------------+-----+--------+------+-----------+
|     formatted_date|delay|distance|origin|destination|
+-------------------+-----+--------+------+-----------+
|1970-01-01 12:45:00|    6|     602|   ABE|        ATL|
|1970-01-02 06:00:00|   -8|     369|   ABE|        DTW|
|1970-01-02 12:45:00|   -2|     602|   ABE|        ATL|
|1970-01-02 06:05:00|   -4|     602|   ABE|        ATL|
|1970-01-03 12:45:00|   -4|     602|   ABE|        ATL|
|1970-01-03 06:05:00|    0|     602|   ABE|        ATL|
|1970-01-04 12:43:00|   10|     602|   ABE|        ATL|
|1970-01-04 06:05:00|   28|     602|   ABE|        ATL|
|1970-01-05 12:45:00|   88|     602|   ABE|        ATL|
|1970-01-05 06:05:00|    9|     602|   ABE|        ATL|
|1970-01-06 12:15:00|   -6|     602|   ABE|        ATL|
|1970-01-06 17:25:00|   69|     602|   ABE|        ATL|
|1970-01-06 12:30:00|    0|     369|   ABE|        DTW|
|1970-01-06 06:25:00|   -3|     602|   ABE|        ATL|
|1970-01-07 06:00:00|    0|     369|   ABE|     

*   Find the days or months when these delays were most common

In [23]:
spark.sql("""
SELECT
  SUBSTRING(date, 1, 2) AS month,
  COUNT(*) AS total_delays
FROM us_delay_flights_tbl
WHERE delay > 0
GROUP BY month
ORDER BY total_delays DESC
""").show()

+-----+------------+
|month|total_delays|
+-----+------------+
|   01|      208606|
|   03|      201046|
|   02|      182075|
+-----+------------+



*   Try to answer with your data results: Were the delays related to winters months or holidays?

Based on the data results, it appears that flight delays can indeed be correlated with winter months. Severe weather conditions like winter storms can lead to increased delays.

## Case clause in Spark SQL
In this query, we are trying to label all US flights, regardless of origin and destination, with an indication of the delays they experienced:


*   Very Long Delays (> 6 hours)
*   Long Delays (2-6 hours)
*   Short Delays (1-2 hours)
*   Tolerable delay (< 1 hour)
We'll add these labels in a new column called: *Flight_Delays*


In [ ]:
spark.sql("""
SELECT delay, origin, destination,
  CASE
    WHEN delay > 360 THEN 'Very Long Delay'
    WHEN delay > 120 AND delay <= 360 THEN 'Long Delay'
    WHEN delay > 60 AND delay <= 120 THEN 'Short Delay'
    WHEN delay > 0 AND delay <= 60 THEN 'Tolenable Delay'
    WHEN delay = 0 THEN 'No Delay'
    ELSE 'Early'
  END AS Flight_Delays
FROM us_delay_flights_tbl
ORDER BY origin, delay DESC
""").show()

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|   Long Delay|
|  305|   ABE|        ATL|   Long Delay|
|  275|   ABE|        ATL|   Long Delay|
|  257|   ABE|        ATL|   Long Delay|
|  247|   ABE|        DTW|   Long Delay|
|  247|   ABE|        ATL|   Long Delay|
|  219|   ABE|        ORD|   Long Delay|
|  211|   ABE|        ATL|   Long Delay|
|  197|   ABE|        DTW|   Long Delay|
|  192|   ABE|        ORD|   Long Delay|
|  180|   ABE|        ATL|   Long Delay|
|  173|   ABE|        DTW|   Long Delay|
|  165|   ABE|        ATL|   Long Delay|
|  159|   ABE|        ATL|   Long Delay|
|  159|   ABE|        ORD|   Long Delay|
|  158|   ABE|        ATL|   Long Delay|
|  151|   ABE|        DTW|   Long Delay|
|  127|   ABE|        ATL|   Long Delay|
|  121|   ABE|        DTW|   Long Delay|
|  118|   ABE|        DTW|  Short Delay|
+-----+------+-----------+-------------+
only showing top

Resturn to slides

## Creating SQL Databases and Tables

In [ ]:
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

DataFrame[]

Creating MANAGED data table

In [ ]:
spark.sql("""
CREATE TABLE managed_us_delay_flights_tbl
  (date STRING, delay INT, distance INT, origin STRING, destination STRING)
""")

DataFrame[]

Creating a UNMANAGED data table:


In [ ]:
spark.sql("""
CREATE TABLE unmanaged_us_delay_flights_tbl
  (date STRING, delay INT, distance INT, origin STRING, destination STRING)
  USING csv OPTIONS (PATH 'mydeparturesdelays.csv')
""")

DataFrame[]

Create a view

In [ ]:
df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")


Selecting from a view (when we do a select into a table, the result is a DF):

In [ ]:
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view")

DataFrame[date: string, delay: int, origin: string, destination: string]

Deleting the view:

In [ ]:
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")


### Viewing the metadata

In [ ]:
spark.catalog.listDatabases()

[Database(name='default', description='Default Hive database', locationUri='file:/content/spark-warehouse'),
 Database(name='learn_spark_db', description='', locationUri='file:/content/spark-warehouse/learn_spark_db.db')]

In [ ]:
spark.catalog.listTables()

[Table(name='managed_us_delay_flights_tbl', database='learn_spark_db', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='unmanaged_us_delay_flights_tbl', database='learn_spark_db', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights_tbl', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [ ]:
spark.catalog.listColumns("managed_us_delay_flights_tbl")

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]